In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-09 21:00:32--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.78MB/s    in 0.2s    

2020-08-09 21:00:33 (4.78 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in kitchen reviews from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

home_df = spark.read.option("header", "true").csv(SparkFiles.get("amazon_reviews_us_Home_v1_00.tsv.gz"), inferSchema=True, sep="\t", timestampFormat="yyyy-mm-dd")

home_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...|2015-01-31 00:08:00|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...| 

## Drop duplicates and incomplete rows

In [6]:
print(f"Initial number of records: {home_df.count()}")
home_df = home_df.dropna()
print(f"Number of records after removing nulls: {home_df.count()}")
home_df = home_df.dropDuplicates()
print(f"Final number of records: {home_df.count()}")

Initial number of records: 6221559
Number of records after removing nulls: 6220614
Final number of records: 6220614


## Examine the schema

In [7]:
home_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create DataFrame to match review_id_table table



In [8]:
# use to_date to convert timestamp to date
from pyspark.sql.functions import to_date

review_id_df = home_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df = review_id_df.withColumn("review_date", to_date("review_date"))

review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R33CZCWZGFNI1P|      12070|B0035EFEOA|     715596773| 2014-01-13|
| RDE2900QKK3T7|      13343|B0014US7BO|     995758202| 2013-01-18|
| R518UELT9I0VA|      13987|B003XDTWN2|     996572389| 2015-01-27|
| RIHOB5CY1CH0N|      14543|B00AC4DOE8|     928755938| 2015-01-05|
| RS06DR03XN9RA|      15802|B00GZWURZA|     605149212| 2014-01-07|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [9]:
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



## Create DataFrame to match products table

In [10]:
# Due to PK constraint unique values have to be selected
# Aggregate and drop count column
products_df = home_df.groupBy("product_id", "product_title").agg({"product_id": "count"}).select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00MO0UHC0|Powerextra Upgrad...|
|B00RCTYLAC|1 X USMC Marine C...|
|B002V9195S|Pro-Mart DAZZ 42-...|
|B00KFWL2JG|Classic Audrey He...|
|B0000635WI|Leachco Snoogle O...|
+----------+--------------------+
only showing top 5 rows



In [11]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



## Create DataFrame to match customers table

In [12]:
from pyspark.sql.types import IntegerType

customers_df = home_df.groupBy('customer_id').agg({'review_id': 'count'}) \
                    .withColumnRenamed('count(review_id)', 'customer_count')
customers_df = customers_df.withColumn('customer_count', customers_df['customer_count'].cast(IntegerType()))

customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     420485|             4|
|    1092226|             2|
|    1327720|             1|
|    1931890|             7|
|    1994559|             1|
+-----------+--------------+
only showing top 5 rows



In [13]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



## Create DataFrame to match vine_table table

In [14]:
vine_df = home_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R33CZCWZGFNI1P|          5|            0|          0|   N|
| RDE2900QKK3T7|          5|            0|          0|   N|
| R518UELT9I0VA|          2|            0|          0|   N|
| RIHOB5CY1CH0N|          5|            0|          0|   N|
| RS06DR03XN9RA|          5|            0|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [15]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



## Write DataFrames to RDS

In [16]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/<database-name>"
config = {"user":"<insert user name>",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}
          

In [17]:
# Write DataFrame to h_review_id_table table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='h_review_id_table', mode=mode, properties=config)

In [18]:
# Write DataFrame to h_products table in RDS
products_df.write.jdbc(url=jdbc_url, table='h_products', mode=mode, properties=config)

In [19]:
# Write DataFrame to h_customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='h_customers', mode=mode, properties=config)

In [20]:
# Write DataFrame to h_vine_table table in RDS
vine_df.write.jdbc(url=jdbc_url, table='h_vine_table', mode=mode, properties=config)